In [1]:
!pip install pyspark
!pip install findspark

Could not fetch URL https://pypi.org/simple/pyspark/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pyspark/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [24]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import requests
import pandas as pd
import json
import time
import gzip

In [25]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [26]:
# Usage of KafkaConsumer class
bootstrap_servers = 'localhost:9092'
topic_name = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

In [27]:
# Connect to local server
client = MongoClient("mongodb://127.0.0.1:27017/")

# Create database called animals
mydb = client["animals"]

# Create Collection (table) called shelterA
collection = mydb.shelterC

In [29]:
from time import sleep
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").getOrCreate()

# Định nghĩa schema cho dữ liệu JSON
json_schema = StructType([
            StructField("reviewId", StringType(), True),
            StructField("placeId", StringType(), True),
            StructField("title", StringType(), True),
            StructField("location/lat", DoubleType(), True),
            StructField("location/lng", DoubleType(), True),
            StructField("categories", StringType(), True),
            StructField("categoryName", StringType(), True),
            StructField("reviewerId", StringType(), True),
            StructField("name", StringType(), True),
            StructField("stars", IntegerType(), True),
            StructField("text", StringType(), True),
            StructField("publishedAtDate", StringType(), True),
            StructField("last_update_time", DoubleType(), True)
            ])

# Đọc dữ liệu từ Kafka topic và chuyển đổi thành DataFrame
kafka_stream_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", topic_name) \
    .load()

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

# Chuyển đổi giá trị từ JSON string sang struct với schema đã định nghĩa
json_stream_df = kafkaDf.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", json_schema).alias("data")) \
    .select("data.*")

x = 0
while True:
    try:
        # Sắp xếp DataFrame theo thời gian cập nhật giảm dần và chỉ lấy dòng đầu tiên
        newest_review_df = json_stream_df.sort(col("last_update_time").desc()).limit(1)
        # newest_review_df = json_stream_df.limit(1)

        # Chuyển DataFrame thành JSON string và lấy giá trị cột 'value'
        message_value = newest_review_df.toJSON().first()

        # In ra giá trị để kiểm tra
        print(message_value)

        # Insert documents (rows) into the database's collection (table)
        collection.insert_one(json.loads(message_value))

        sleep(2)
        clear_output(wait=True)

    except KeyboardInterrupt:
        print("break")
        break

print("Live view ended...")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 977.0 failed 1 times, most recent failure: Lost task 0.0 in stage 977.0 (TID 969) (192.168.241.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.